In [21]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 325.1 kB/s eta 0:07:42
   ---------------------------------------- 0.2/150.0 MB 1.1 MB/s eta 0:02:11
   ---------------------------------------- 0.2/150.0 MB 1.1 MB/s eta 0:02:11
   ---------------------------------------- 0.3/150.0 MB 1.6 MB/s eta 0:01:33
   ---------------------------------------- 0.4/150.0 MB 1.5 MB/s eta 0:01:43
   ---------------------------------------- 0.6/150.0 MB 1.8 MB/s eta 0:01:22
   ---------------------------------------- 0.7/150.0 MB 2.1 MB/s eta 0:01:12
   ---------------------------------------- 0.9/150.0 MB 2.3 MB/s eta 0:01:06
   ---------------------------------------- 1.1/150.0 MB 2.5 MB/s eta 0:01:01
   ---------------------------------------- 1.3/150.0 MB 2.8 MB/s eta 0:00:55
   ---------------------------------------- 1.4/150.0 MB 2.8 MB/s eta 0:00:5

In [1]:
# Importing usefull libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [23]:
# Importing Energy data
EU_energy_data = pd.read_csv(f"./time_series/time_series_60min_singleindex.csv")

In [24]:
EU_energy_data.head()

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,BE_solar_generation_actual,...,SI_load_actual_entsoe_transparency,SI_load_forecast_entsoe_transparency,SI_solar_generation_actual,SI_wind_onshore_generation_actual,SK_load_actual_entsoe_transparency,SK_load_forecast_entsoe_transparency,SK_solar_generation_actual,SK_wind_onshore_generation_actual,UA_load_actual_entsoe_transparency,UA_load_forecast_entsoe_transparency
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,5946.0,6701.0,35.0,NaN,69.0,9484.0,9897.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,5726.0,6593.0,45.0,NaN,64.0,9152.0,9521.0,NaN,...,1045.47,816.0,NaN,1.17,2728.0,2860.0,3.8,NaN,NaN,NaN
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,5347.0,6482.0,41.0,NaN,65.0,8799.0,9135.0,NaN,...,1004.79,805.0,NaN,1.04,2626.0,2810.0,3.8,NaN,NaN,NaN
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,5249.0,6454.0,38.0,NaN,64.0,8567.0,8909.0,NaN,...,983.79,803.0,NaN,1.61,2618.0,2780.0,3.8,NaN,NaN,NaN


### Analysing Energy data of Austria

In [25]:
austria_df = EU_energy_data[[
    'utc_timestamp',
    'AT_load_actual_entsoe_transparency',
    'AT_load_forecast_entsoe_transparency',
    'AT_price_day_ahead',
    'AT_solar_generation_actual',
    'AT_wind_onshore_generation_actual'
]].copy()

### Preprocessing the data

In [26]:
print(austria_df.isnull().sum())

utc_timestamp                               0
AT_load_actual_entsoe_transparency          1
AT_load_forecast_entsoe_transparency        1
AT_price_day_ahead                      17556
AT_solar_generation_actual                 62
AT_wind_onshore_generation_actual          49
dtype: int64


In [ ]:
austria_df.ffill(inplace=True)
austria_df.bfill(inplace=True)

C:\Users\rande\AppData\Local\Temp\ipykernel_13728\2383035845.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  austria_df.fillna(method='ffill', inplace=True)
C:\Users\rande\AppData\Local\Temp\ipykernel_13728\2383035845.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  austria_df.fillna(method='bfill', inplace=True)


In [28]:
print(austria_df.isnull().sum())

utc_timestamp                           0
AT_load_actual_entsoe_transparency      0
AT_load_forecast_entsoe_transparency    0
AT_price_day_ahead                      0
AT_solar_generation_actual              0
AT_wind_onshore_generation_actual       0
dtype: int64


In [29]:
austria_df['utc_timestamp'] = pd.to_datetime(austria_df['utc_timestamp'])

In [ ]:
print(austria_df.info())

        load_actual  load_forecast         price  solar_generation  \
count  50401.000000   50401.000000  50401.000000      50401.000000   
mean    7070.126327    7039.841829     43.674491        135.114502   
std     1400.258228    1355.370232     18.006037        198.888969   
min      664.000000    3739.000000    -76.000000          0.000000   
25%     5941.000000    5941.000000     29.000000          0.000000   
50%     7032.000000    7018.000000     42.000000         17.000000   
75%     8079.000000    7986.000000     62.000000        224.000000   
max    10803.000000   11608.000000    187.000000       1166.000000   

       Wind_generation          Hour      load_t-1     load_t-24  \
count     50401.000000  50401.000000  50401.000000  50401.000000   
mean        729.287157     11.500228   7070.137259   7069.651078   
std         691.767267      6.922376   1400.247301   1400.291140   
min           0.000000      0.000000    664.000000    664.000000   
25%         163.000000      6

### Creating more features

In [30]:
austria_df['Weekday'] = austria_df['utc_timestamp'].dt.day_name()
austria_df['Hour'] = austria_df['utc_timestamp'].dt.hour
austria_df['Month'] = austria_df['utc_timestamp'].dt.month_name()

In [31]:
# Clean the data
austria_df = austria_df.set_index('utc_timestamp')
austria_df = austria_df.sort_index()

austria_df.columns = austria_df.columns.str.strip()
austria_df.rename(columns={'AT_load_actual_entsoe_transparency': 'load_actual','AT_load_forecast_entsoe_transparency':'load_forecast','AT_price_day_ahead':'price','AT_solar_generation_actual':'solar_generation','AT_wind_onshore_generation_actual':'Wind_generation'}, inplace=True)

### t-1 gives short-term momentum.
### t-24 captures daily cycles.

In [32]:
austria_df['load_t-1'] = austria_df['load_actual'].shift(1)     # 1 hour ago
austria_df['load_t-24'] = austria_df['load_actual'].shift(24)   # 24 hours ago

# Optional: Add rolling average
austria_df['load_rolling_24h'] = austria_df['load_actual'].rolling(window=24).mean()
austria_df.head()

,load_actual,load_forecast,price,solar_generation,Wind_generation,Weekday,Hour,Month,load_t-1,load_t-24,load_rolling_24h
utc_timestamp,,,,,,,,,,,
2014-12-31 23:00:00+00:00,5946.0,6701.0,35.0,2.0,69.0,Wednesday,23,December,NaN,NaN,NaN
2015-01-01 00:00:00+00:00,5946.0,6701.0,35.0,2.0,69.0,Thursday,0,January,5946.0,NaN,NaN
2015-01-01 01:00:00+00:00,5726.0,6593.0,45.0,2.0,64.0,Thursday,1,January,5946.0,NaN,NaN
2015-01-01 02:00:00+00:00,5347.0,6482.0,41.0,2.0,65.0,Thursday,2,January,5726.0,NaN,NaN
2015-01-01 03:00:00+00:00,5249.0,6454.0,38.0,2.0,64.0,Thursday,3,January,5347.0,NaN,NaN


In [ ]:
austria_df.ffill(inplace=True)
austria_df.bfill(inplace=True)
austria_df.head()

C:\Users\rande\AppData\Local\Temp\ipykernel_13728\1861709718.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  austria_df.fillna(method='ffill', inplace=True)
C:\Users\rande\AppData\Local\Temp\ipykernel_13728\1861709718.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  austria_df.fillna(method='bfill', inplace=True)


,load_actual,load_forecast,price,solar_generation,Wind_generation,Weekday,Hour,Month,load_t-1,load_t-24,load_rolling_24h
utc_timestamp,,,,,,,,,,,
2014-12-31 23:00:00+00:00,5946.0,6701.0,35.0,2.0,69.0,Wednesday,23,December,5946.0,5946.0,6310.125
2015-01-01 00:00:00+00:00,5946.0,6701.0,35.0,2.0,69.0,Thursday,0,January,5946.0,5946.0,6310.125
2015-01-01 01:00:00+00:00,5726.0,6593.0,45.0,2.0,64.0,Thursday,1,January,5946.0,5946.0,6310.125
2015-01-01 02:00:00+00:00,5347.0,6482.0,41.0,2.0,65.0,Thursday,2,January,5726.0,5946.0,6310.125
2015-01-01 03:00:00+00:00,5249.0,6454.0,38.0,2.0,64.0,Thursday,3,January,5347.0,5946.0,6310.125
